In [48]:
import pandas as pd
from models import FinNewsArticle
from typing import List
import chromadb
from chromadb.utils import embedding_functions
import openai
import psutil
from datetime import datetime
import pytz
import json
from openai import OpenAI

In [13]:
def load_financial_news_data(csv_path: str = 'russian_fin_news/mini_df.csv') -> List[FinNewsArticle]:
    """
    Загружает данные финансовых новостей из CSV и преобразует в список Pydantic объектов.
    
    Args:
        csv_path: Путь к CSV файлу с данными
        
    Returns:
        Список объектов FinNewsArticle
        
    Raises:
        FileNotFoundError: Если файл не найден
        ValidationError: Если данные не соответствуют схеме
    """
    try:
        df = pd.read_csv(csv_path)
        return [FinNewsArticle(**row) for row in df.to_dict('records')]
    except FileNotFoundError:
        raise FileNotFoundError(f"Файл {csv_path} не найден")
    except Exception as e:
        raise ValueError(f"Ошибка при обработке данных: {e}")

In [14]:
# Тестируем функцию загрузки данных
articles = load_financial_news_data()

print(f"Загружено статей: {len(articles)}")
print(f"Первая статья: {articles[0].id} - {articles[0].sphere}")
print(f"Диапазон оценок: {min(article.answer for article in articles)} до {max(article.answer for article in articles)}")

Загружено статей: 150
Первая статья: 0 - Финансы
Диапазон оценок: -75.0 до 85.0


In [32]:
def get_or_create_collection(articles: List[FinNewsArticle], openai_api_key: str, collection_name: str = "financial_news", persist_directory: str = "./chroma_db") -> chromadb.Collection:
    """
    Получает существующую коллекцию или создает новую, если её нет.
    
    Args:
        articles: Список объектов FinNewsArticle
        openai_api_key: API ключ OpenAI
        collection_name: Название коллекции в ChromaDB
        persist_directory: Директория для сохранения базы данных
        
    Returns:
        ChromaDB коллекция с эмбедингами
    """
    # Создаем OpenAI embedding function
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
        api_key=openai_api_key,
        model_name="text-embedding-3-small"
    )

    # Создаем персистентный клиент
    client = chromadb.PersistentClient(path=persist_directory)

    try:
        # Пытаемся получить существующую коллекцию
        collection = client.get_collection(
            name=collection_name,
            embedding_function=openai_ef
        )
        print(f"Загружена существующая коллекция '{collection_name}' с {collection.count()} документами")
        return collection

    except Exception:  # Ловим любое исключение, если коллекция не найдена
        # Коллекция не существует, создаем новую
        print(f"Создание новой коллекции '{collection_name}'...")

        collection = client.create_collection(
            name=collection_name,
            embedding_function=openai_ef
        )

        # Подготавливаем данные для эмбедингов
        documents = []
        metadatas = []
        ids = []

        for article in articles:
            document = f"{article.reasoning} {article.article_text} {article.sphere}"
            documents.append(document)

            metadata = {
                "id": article.id,
                "answer": article.answer,
                "source": article.source,
                "date": article.date.isoformat()
            }
            metadatas.append(metadata)
            ids.append(str(article.id))

        # Добавляем документы в коллекцию
        collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )

        print(f"Коллекция создана с {collection.count()} документами")
        return collection

In [ ]:
# Тестируем создание коллекции эмбедингов
OPENAI_API_KEY = "****"
collection = get_or_create_collection(articles, OPENAI_API_KEY)

print(f"Создана коллекция с {collection.count()} документами")
print(f"Название коллекции: {collection.name}")

# Проверяем первый документ
result = collection.get(ids=["0"], include=["metadatas", "documents"])
print(f"Пример метаданных: {result['metadatas'][0]}")
print(f"Начало документа: {result['documents'][0][:100]}...")

Создание новой коллекции 'financial_news'...
Коллекция создана с 150 документами
Создана коллекция с 150 документами
Название коллекции: financial_news
Пример метаданных: {'date': '2025-02-21', 'source': 'Интерфакс', 'id': 0, 'answer': 15.0}
Начало документа: Статья о финансовых результатах Rivian, американского производителя электромобилей. Хотя Rivian - не...


In [ ]:
def get_system_stats():
    """
    Получает статистику загрузки системы: CPU и память.
    
    Returns:
        dict: Словарь с информацией о CPU и памяти
    """
    cpu_percent = psutil.cpu_percent(interval=1)
    memory = psutil.virtual_memory()

    return {
        "cpu_percent": cpu_percent,
        "memory_percent": memory.percent,
        "memory_available_gb": round(memory.available / (1024**3), 2),
        "memory_total_gb": round(memory.total / (1024**3), 2)
    }
get_system_stats()

{'cpu_percent': 46.0,
 'memory_percent': 86.0,
 'memory_available_gb': 1.12,
 'memory_total_gb': 8.0}

In [ ]:
def get_moscow_time():
    """
    Получает текущее время в Москве.
    
    Returns:
        str: Текущее время в Москве в формате строки
    """
    moscow_tz = pytz.timezone('Europe/Moscow')
    moscow_time = datetime.now(moscow_tz)

    return moscow_time.strftime("%Y-%m-%d %H:%M:%S %Z")
get_moscow_time()

'2025-08-02 14:28:24 MSK'

In [43]:
def search_similar_articles(query: str, collection: chromadb.Collection, top_k: int = 5) -> dict:
    """
    Ищет наиболее похожие статьи по запросу используя векторный поиск.
    
    Args:
        query: Текст запроса для поиска
        collection: ChromaDB коллекция с эмбедингами
        top_k: Количество топовых результатов (по умолчанию 5)
        
    Returns:
        dict: Результаты поиска с документами, метаданными и расстояниями
    """
    results = collection.query(
        query_texts=[query],
        n_results=top_k,
        include=["documents", "metadatas", "distances"]
    )

    # Форматируем результаты для удобства
    formatted_results = []
    for i in range(len(results['documents'][0])):
        result = {
            "id": results['metadatas'][0][i]['id'],
            "distance": results['distances'][0][i],
            "answer": results['metadatas'][0][i]['answer'],
            "source": results['metadatas'][0][i]['source'],
            "date": results['metadatas'][0][i]['date'],
            "document": results['documents'][0][i][:200] + "..."  # Первые 200 символов
        }
        formatted_results.append(result)

    return {
        "query": query,
        "total_results": len(formatted_results),
        "results": formatted_results
    }

In [47]:
# Пример использования функции поиска
query = "Информация о Промсвязьбанке"

# Выполняем поиск
search_results = search_similar_articles(query, collection, top_k=5)

# Выводим результаты
print(f"Запрос: {search_results['query']}")
print(f"Найдено результатов: {search_results['total_results']}\n")

for i, result in enumerate(search_results['results'], 1):
    print(f"Результат {i}:")
    print(f"  ID: {result['id']}")
    print(f"  Оценка влияния: {result['answer']}")
    print(f"  Источник: {result['source']}")
    print(f"  Дата: {result['date']}")
    print(f"  Схожесть: {1 - result['distance']:.3f}")  # Чем ближе к 1, тем больше схожесть
    print(f"  Текст: {result['document']}")
    print("-" * 80)

Запрос: Информация о Промсвязьбанке
Найдено результатов: 5

Результат 1:
  ID: 5
  Оценка влияния: 25.0
  Источник: Интерфакс
  Дата: 2025-02-21
  Схожесть: 0.143
  Текст: Статья описывает судебное решение о взыскании с Ost-West Handelsbank (бывшего VTB Bank Europe) в пользу Промсвязьбанка значительной суммы. Это событие может положительно повлиять на акции Промсвязьбан...
--------------------------------------------------------------------------------
Результат 2:
  ID: 126
  Оценка влияния: 25.0
  Источник: Интерфакс
  Дата: 2025-02-19
  Схожесть: -0.236
  Текст: Снижение ставок по вкладам Сбербанка может привести к перетоку средств с депозитов в другие финансовые инструменты, что потенциально положительно повлияет на рынок ценных бумаг и кредитование. Однако ...
--------------------------------------------------------------------------------
Результат 3:
  ID: 127
  Оценка влияния: -60.0
  Источник: Интерфакс
  Дата: 2025-02-19
  Схожесть: -0.257
  Текст: Статья сообщает о сохранени

In [53]:
def call_llm_with_tools(query: str, collection: chromadb.Collection, openai_api_key: str) -> str:
    """
    Отправляет запрос к LLM с возможностью вызова одной из трех функций.
    
    Args:
        query: Запрос пользователя
        collection: ChromaDB коллекция для RAG поиска
        openai_api_key: API ключ OpenAI
        
    Returns:
        str: Ответ LLM с результатами выполнения функций
    """
    client = OpenAI(api_key=openai_api_key)

    # Определяем доступные инструменты
    tools = [
        {
            "type": "function",
            "function": {
                "name": "search_financial_news",
                "description": "Поиск по базе российских финансовых новостей. Используйте для вопросов о финансах, экономике, компаниях, рынках.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Поисковый запрос для поиска релевантных финансовых новостей"
                        },
                        "top_k": {
                            "type": "integer",
                            "description": "Количество результатов (по умолчанию 5)",
                        }
                    },
                    "required": ["query", "top_k"],
                    "additionalProperties": False
                },
                "strict": True
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_system_stats",
                "description": "Получить статистику загрузки системы: CPU и память. Используйте для вопросов о производительности системы.",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "additionalProperties": False
                },
                "strict": True
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_moscow_time",
                "description": "Получить текущее время в Москве. Используйте для вопросов о времени в Москве.",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    ]

    # Первый вызов LLM
    messages = [
        {
            "role": "system",
            "content": "Вы - полезный ассистент, который может искать информацию в базе финансовых новостей, получать системную статистику и время в Москве. Выберите подходящую функцию на основе запроса пользователя."
        },
        {
            "role": "user",
            "content": query
        }
    ]

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    # Проверяем, были ли вызваны функции
    if response.choices[0].message.tool_calls:
        # Добавляем сообщение модели в историю
        messages.append(response.choices[0].message)

        # Выполняем вызовы функций
        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)

            # Вызываем соответствующую функцию
            if function_name == "search_financial_news":
                result = search_similar_articles(
                    function_args["query"],
                    collection,
                    function_args.get("top_k", 5)
                )
                function_result = json.dumps(result, ensure_ascii=False, indent=2)

            elif function_name == "get_system_stats":
                result = get_system_stats()
                function_result = json.dumps(result, ensure_ascii=False, indent=2)

            elif function_name == "get_moscow_time":
                result = get_moscow_time()
                function_result = result

            else:
                function_result = "Неизвестная функция"

            # Добавляем результат функции в сообщения
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": function_result
            })

        # Второй вызов LLM для генерации финального ответа
        final_response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages,
            tools=tools
        )

        return final_response.choices[0].message.content

    else:
        # Если функции не вызывались, возвращаем обычный ответ
        return response.choices[0].message.content

In [56]:
# Пример использования
# result = call_llm_with_tools("Какое сейчас время в Москве?", collection, OPENAI_API_KEY)
# print(result)

# result = call_llm_with_tools("Покажи загрузку системы", collection, OPENAI_API_KEY)
# print(result)

result = call_llm_with_tools("Найди новости о банка СБЕРБАНК", collection, OPENAI_API_KEY)
print(result)

Вот последние новости о Сбербанке:

1. Снижение ставок по вкладам Сбербанка может привести к перетоку средств с депозитов в другие финансовые инструменты, что может положительно повлиять на рынок ценных бумаг и кредитование. (Интерфакс, 19 февраля 2025)

2. Также снижение ставок по вкладам может привести к снижению спроса на рубли, что негативно скажется на стоимости акций российских банков. (Интерфакс, 19 февраля 2025)

Если нужно, могу подробнее рассказать по каждой новости.
